In [ ]:
SURVEY_SUBMODULE_DIR = "./survey" # this notebook will create a folder "explanations" there
SAVE_PATH = "./dataset_user_study.csv" # created by document_selection_user_study.ipynb

This notebook exports the user-study csv from `SAVE_PATH` to the UI. The UI uses a JSON file for each document and explainer-detector pair. The explanations are stored as HTML files. For LIME and SHAP, javascript is bundled, for Anchor, it must be imported due to bundle size. 

In [ ]:
from gpt2outputdataset.detector_radford import DetectorRadford
from detectgpt.detector_detectgpt import DetectorDetectGPT
from detector_dummy import DetectorDummy
from detector_guo import DetectorGuo
from explainer_wrappers import LIME_Explainer, SHAP_Explainer, Anchor_Explainer
from IPython.core.display import display, HTML
from tqdm import tqdm

In [ ]:
explainer_classes = [LIME_Explainer, SHAP_Explainer, Anchor_Explainer]
detector_classes = [ DetectorGuo, DetectorRadford, DetectorDetectGPT]

In [ ]:
import pandas as pd
import json
import os

In [ ]:
df = pd.read_pickle("./dataset_user_study_new.pkl")


In [ ]:
df.groupby(["Detector"]).count()

In [ ]:
if not os.path.exists(os.path.join(SURVEY_SUBMODULE_DIR)): 
    raise FileNotFoundError("Make sure you cloned the survey submodule")
if not os.path.exists(os.path.join(SURVEY_SUBMODULE_DIR, "explanations")): 
    os.makedirs(os.path.join(SURVEY_SUBMODULE_DIR, "explanations"))
if not os.path.exists(os.path.join(SURVEY_SUBMODULE_DIR, "explanations", "html")): 
    os.makedirs(os.path.join(SURVEY_SUBMODULE_DIR, "explanations", "html"))
if not os.path.exists(os.path.join(SURVEY_SUBMODULE_DIR, "explanations", "data")): 
    os.makedirs(os.path.join(SURVEY_SUBMODULE_DIR, "explanations", "data"))

While the explanations are cached, this will still take some time as `detector.predict_proba` must be called.

In [ ]:

for detector_name, group in df.groupby("Detector"):
    if detector_name == "DetectorDetectGPT":
        detector = DetectorDetectGPT()
    if detector_name == "DetectorRadford":
        detector = DetectorRadford()
    if detector_name == "DetectorGuo":
        detector = DetectorGuo()

    documents_group = list(group.reset_index().iterrows()) # reset_index: count from 0 in each group
    
    for explainer_class in explainer_classes:
        explainer = explainer_class(detector)
        for index, row in tqdm(documents_group, "Exporting " + detector.__class__.__name__ + " " + explainer.__class__.__name__):
            a = row["Documents Phases 1+3"]
            b = row["Documents Phases 2+4"]
            # Phase 1 + 3: 
            path_explanation_html = os.path.join(SURVEY_SUBMODULE_DIR, "explanations", "html", explainer.get_hash(a)+".html")
            with open(path_explanation_html, "w", encoding="UTF-8") as text_file:
                text_file.write(explainer.get_HTML(a, bundle=False))
            # print(row)
            # display(HTML(explainer.get_HTML(a)))
            # display(HTML(explainer.get_HTML(b)))
            # print("----------------")
            path_explanation_json = os.path.join(SURVEY_SUBMODULE_DIR, "explanations", "data", explainer.get_hash(a)+".json")
            with open(path_explanation_json, "w", encoding="UTF-8") as text_file:
                explanation_data = { # do not include explanation
                    "document_nr": index,
                    "detector" : detector.__class__.__name__,
                    "explainer" : explainer.__class__.__name__,
                    "ground_truth" : row["GT a"],
                    "detector_label" : int(detector.predict_label([row["Documents Phases 1+3"]])[0]),
                    "detector_p_machine": float(detector.predict_proba([row["Documents Phases 1+3"]])[0][0]),
                    "detector_p_human": float(detector.predict_proba([row["Documents Phases 1+3"]])[0][1]),
                    "document": row["Documents Phases 1+3"],
                    "explanation_filename": explainer.get_hash(a),               
                }
                text_file.write(json.dumps(explanation_data))
            # Phase 2+4: Just string
            path_explanation_html = os.path.join(SURVEY_SUBMODULE_DIR, "explanations", "html", explainer.get_hash(b)+".html")
            #  display(HTML(explainer.get_HTML(b)))
            path_explanation_json = os.path.join(SURVEY_SUBMODULE_DIR, "explanations", "data", explainer.get_hash(b)+".json")
            with open(path_explanation_json, "w", encoding="UTF-8") as text_file:
                explanation_data = { # do not include explanation
                    "document_nr": index,
                    "detector" : detector.__class__.__name__,
                    "explainer" : explainer.__class__.__name__,
                    "document": row["Documents Phases 2+4"]                
                }
                text_file.write(json.dumps(explanation_data))
        # break    